# Metadata Refresh for SQL Server (Project Setup)

This notebook contains the SQL Set-up scripts for the eltSnap project "Metadata Refresh for SQL Server"

These script needs to be run **after** ingesting the notebook called **Ingest ADS Notebook Files (Project)**

> To run these SQL scripts, first connect to your SQL Server instance of **eltsnap_v2**

## Add a Connection for the SQL Server where you want to Extract Metadata (Source)

Verify the Parameter Values in the EXEC statement below. You may need to _replace_ 'localhost' with your _server_name_

> Note: Initial Parameter Values can be substituted at runtime

In [ ]:
EXEC [elt].[Save Connection by Name] 
     @connection_name = 'sql_server'
   , @server_name = 'localhost'
   , @database_name = 'master'
   , @provider = 'SQL Server Native Client 11.0'

## Import the "Notebook Based" Project into eltSnap

This will refresh the cooresponding logic used by eltSnap based on the _Ingested_ Notebook code

In [ ]:
EXEC [ads].[Import Project from ADS Notebooks] 'Metadata Refresh v2', 'notebook_snap_basics\content\metadata_refresh_for_sql_server_project.ipynb';

## Alternate (Custom) Project Import

You have the option to specify _multiple source files_, along with _custom sequence numbers_ when importing a Project. This feature allows for:
- Projects to be defined by pulling code from 2 or more _file based_ resources (this feature currently supports **.ipynb** and **.sql** files which contain notebook snap _metadata_)
- Parallel package execution by specifing the _same sequence number_ for 2 or more packages

The following script can be used to populate the table: [ads].[project_cells] with the desired sequence numbers and file resources, along with the command used to Import the custom defined project

> Be sure to first **uncomment** the code cell below before running

In [ ]:
/*
DELETE FROM [ads].[project_cells] WHERE [project_name] = 'Metadata Refresh v2'

INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'dim delete - dbo - dim_column', 510, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'dim delete - dbo - dim_table', 510, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Insert Metadata Server Name', 110, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Merge stg to dbo schema - dim_column', 420, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Merge stg to dbo schema - dim_table', 420, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Merge stg to meta schema - dim_column', 410, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Merge stg to meta schema - dim_database', 220, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Merge stg to meta schema - dim_table', 410, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Refresh Database Names', 210, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'src to stg - SQL Server columns', 310, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'src to stg - SQL Server tables', 310, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Metadata Refresh v2', N'notebook_snap_basics\content\metadata_refresh_for_sql_server.ipynb', N'Truncate STG tables', 140, NULL)

EXEC [ads].[Import Project from ADS] 'Metadata Refresh v2'
*/